In [1]:
import numpy as np
import pandas as pd
df = pd.read_csv("btcusdt_1m.csv")
df.head()



,Open time,Open,High,Low,Close,Volume,Close time,Quote asset volume,Number of trades,Taker buy base asset volume,Taker buy quote asset volume,Ignore
0,2023-12-13 00:00:00,41492.38,41506.00,41476.17,41476.18,23.92517,2023-12-13 00:00:59.999,9.927303e+05,1003,9.64939,400429.352380,0
1,2023-12-13 00:01:00,41476.18,41479.51,41435.13,41441.32,42.52961,2023-12-13 00:01:59.999,1.763006e+06,1254,6.92112,286888.931824,0
2,2023-12-13 00:02:00,41441.32,41441.33,41394.27,41394.28,78.02927,2023-12-13 00:02:59.999,3.232068e+06,1523,10.56972,437816.932378,0
3,2023-12-13 00:03:00,41394.27,41421.26,41394.27,41421.26,23.38614,2023-12-13 00:03:59.999,9.683890e+05,1037,13.11932,543220.585933,0
4,2023-12-13 00:04:00,41421.26,41430.44,41416.59,41424.95,14.50482,2023-12-13 00:04:59.999,6.008739e+05,822,4.46984,185163.656065,0
